# Lecture 20: Advanced Image Processing with Machine Learning

Welcome to the second part of our exploration into Image Processing with Machine Learning. In this session, we dive deeper into practical applications using the CIFAR-10 dataset. We'll explore various preprocessing techniques and their impact on model performance, offering a hands-on approach to understanding these concepts.


### Set up imports

In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from skimage.color import rgb2gray
from skimage.filters import sobel

### Load images from CIFAR-10 Dataset

The CIFAR-10 dataset is a collection of images that are commonly used to train machine learning and computer vision algorithms. It is one of the most widely used datasets for machine learning research. The CIFAR-10 dataset contains 60,000 32x32 color images in 10 different classes. [Learn more about CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html).

In [ ]:
# use this function to load the CIFAR-10 dataset from the data folder
def load_cifar_batch(filename):
    """Load a single batch of CIFAR-10."""
    with open(filename, 'rb') as file:
        # The encoding 'bytes' is required for Python 3 compatibility
        batch = pickle.load(file, encoding='bytes')
        images = batch[b'data']
        labels = batch[b'labels']
        # Reshape the images: the dataset is flattened, so you need to reshape it to 32x32x3
        images = images.reshape((len(images), 3, 32, 32)).transpose(0, 2, 3, 1)
        labels = np.array(labels)
        return images, labels

In [ ]:
# load batch 1 of CIFAR-10


### Inspect images and labels

Understanding the structure and format of our dataset is crucial. Let’s start by examining the lengths of images and labels to get a sense of the dataset's size.

In [ ]:
# look at length of images and labels


### Inspect the first three images

Visualizing our data is just as important as understanding its structure. Let’s display the first few images from our dataset along with their corresponding labels to see what we’re working with.


In [ ]:
# Function to show an image

# Show the first three images



### Define labels
Each image in CIFAR-10 is associated with a label from 10 classes. Here, we define a list of label names to make our data more understandable.


In [ ]:
label_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


### Inspect the first label

To further familiarize ourselves with the dataset, let's inspect the label of the first image. This step helps us connect an image with its categorical representation.


In [ ]:
# inspect first label


### Preprocess data

Data preprocessing is a critical step in any machine learning workflow. Here, we'll normalize pixel values to improve our model's convergence during training. We'll also reshape the data to fit our model's input requirements.


In [ ]:
# Normalize pixel values to be between 0 and 1


In [ ]:
# inspect the shape of the images


In [ ]:
# Flatten the images


### Splitting Dataset into Training and Test Sets

In [ ]:
# Split the data into training and testing sets


### Training a Machine Learning Model

In [ ]:
# Initialize the model


# Train the model


### Evaluating the Model
After training the model, evaluate its performance on the test set.

In [ ]:
# Make predictions


# Calculate accuracy



### Activity: Preprocessing Techniques and Model Performance

Experiment with different preprocessing techniques. Assess how each technique affects the performance of a simple image classification model trained on the CIFAR-10 dataset. Reflect on your findings.


### Grayscale Preprocessing
Converting images to grayscale simplifies our model by reducing the dimensionality of the input data. Let’s preprocess our images into grayscale and evaluate how this impacts model performance.


Use this code to create an array of grayscale image data
```python
images_gray = rgb2gray(images)
```

In [ ]:
# create an array of grayscale images


In [ ]:
assert images_gray.shape == (10000, 32, 32)

In [ ]:
# inspect the first image


In [ ]:
# flatten the images to make them ready for the model 


In [ ]:
assert images_gray_flattened.shape == (10000, 1024)

### Grayscale model training
Perform a train test split and train a random forest model with 100 estimators and a random state of 42

In [ ]:
# do train test split


# initialize the model


# train the model


### Evaluate grayscale model
Use accuracy score to evaluate how preprocessing the data affects model performance

In [ ]:
# Make predictions


# Calculate accuracy


In [ ]:
assert accuracy == 0.373

### Use edge detection to preprocess images
Use scikit-image's [sobel](https://en.wikipedia.org/wiki/Sobel_operator) function to emphasize the edges of the grayscale images

```python
sobel_image = sobel(image)
```

In [ ]:
# write a list comprehension to create a list of grayscale edge-detected images


### Convert the list back to a numpy array

```python
images_edges = np.array(images_edges)
```

In [ ]:
# convert to numpy array


In [ ]:
# inspect the first image


In [ ]:
# inspect image_edges data


In [ ]:
# flatten the images to make them ready for the model 


### Train a random forest with 100 estimators and random state = 42

In [ ]:
assert accuracy_edges == 0.318